## PCA for Image Compression

The goal of this exercise is to use PCA for image compression in Python.

##### a) Load an image of the famous painting "The Starry Night" by Vincent Van Gogh, and store it as an RGB-image.

In [ ]:
# For easy image loading, we use skimage.io via io.imread(url)
# You can get it via installing the scikit-image module/package
# e.g. by executing this command from a cell -->   %pip install scikit-image

import numpy as np
from skimage import io
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
# load the image
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1364px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg"
img = io.imread(url)

# save it locally
io.imsave("original.png", img)

In [ ]:
# note, that the image is stored as a 3-dimensional numpy array
print("image data type:", type(img))
print("image shape:", img.shape)

In [ ]:
# the third axis specifies the "color channels" of the image,
# split each channel into a seperate variable
r = img[:, :, 0]
g = img[:, :, 1]
b = img[:, :, 2]

In [ ]:
# have a look at the original image and the single channels

# prepare pyplot figure
fig = plt.figure(figsize=(16, 16))

# draw original image
fig.add_subplot(2, 1, 1)
plt.title("original")
plt.imshow(img)

# draw reds
fig.add_subplot(2, 3, 4)
plt.title("red")
plt.imshow(r, cmap="Greys")

# draw greens
fig.add_subplot(2, 3, 5)
plt.title("green")
plt.imshow(g, cmap="Greys")

# draw blues
fig.add_subplot(2, 3, 6)
plt.title("blue")
plt.imshow(b, cmap="Greys")

plt.show()

##### b) Apply PCA to compress the image.

In [ ]:
def compress_image(image, number_of_components=None, keep_pca_objects=False):
  """
  Uses PCA to compress a multi-channel image.

  Parameters
  ----------
  image : numpy.ndarray
    The image to compress
  number_of_components : int, optional
    The number of principal components to use for compression. If None, keeps all
  keep_pca_objects : bool, optional
    If True, a list of PCA objects will be returned alongside the compressed
    image. The default is False.

  Returns
  -------
  compressed_image : numpy.ndarray
  pca_objects : list, optional (see argument keep_pca_objects)
  """
  list_of_compressed_channels = list()
  list_of_pca_objects = list()
  
  # for each channel of the image
  for i in range(image.shape[2]):
    # get channel image, normalize data to [0, 1] before applying PCA
    channel_data = img[:, :, i] / 255.0
    
    # perform pca
    pca = PCA(n_components=number_of_components)
    pca.fit(channel_data)
    compressed_channel_data = pca.inverse_transform(pca.transform(channel_data))
    
    # save fitted pca object
    if keep_pca_objects:
      list_of_pca_objects.append(pca)
    
    # save compressed channel
    list_of_compressed_channels.append(compressed_channel_data)
  
  # combine compressed channels into one compressed multi-channel image
  compressed_image = np.stack(list_of_compressed_channels, axis=2)
  
  # after PCA, the values may fall out of range [0, 255], and need to be
  #  renormalized accordingly (alternative: clipping)
  compressed_image = (compressed_image - np.min(compressed_image))
  compressed_image = (compressed_image * 255 / np.max(compressed_image))  \
                     .astype(np.uint8)
                     
  returns = None
  if keep_pca_objects:
    returns = compressed_image, list_of_pca_objects
  else:
    returns = compressed_image
  return returns

In [ ]:
# use the custom function to compress the original image using 5 principal components
cimg = compress_image(img, 5)
plt.title("pca_compression_5_components")
plt.imshow(cimg);

##### c) Apply PCA for $n\in\{1, 2, 5, 10, 20\}$. If your computer if powerful enough, you can increase even further. Plot and save the compressed image in each iteration. Observe the size of the image files. Looking at the images, at what point do you clearly identify the painting?

In [ ]:
for n in [1, 2, 5, 10, 20]:
  cimg = compress_image(img, n)
  io.imsave("pca_compression_"+str(n)+"_components.png", cimg)

##### d) Determine a reasonable number of clusters using the "elbow criterion". For this purpose, create a scree plot that plots the explained variance of each component against the number of components, e.g. for $n\in[1, ..., 10]$. Does the "elbow point" correspond to your visual impression in c) ?

In [ ]:
fig = plt.figure(figsize=(18, 6))
n = 10
x = np.linspace(1, n, n)

cimg, pca = compress_image(img, number_of_components=n, keep_pca_objects=True)
plt.xticks(ticks=x)
plt.plot(x, pca[0].explained_variance_ratio_, color="red")
plt.plot(x, pca[1].explained_variance_ratio_, color="green")
plt.plot(x, pca[2].explained_variance_ratio_, color="blue")
plt.plot(x, pca[0].explained_variance_ratio_ \
            + pca[1].explained_variance_ratio_ \
            + pca[2].explained_variance_ratio_, color="black")

plt.show()

# elbow points for n=2 or n=3 (ambiguous). 